# M2

#### Import Neccessary Packages
The following packages included in cb1020.yml are necessary for the model to run. 
These include classes of the mesa package, as well as the agent classes improted from the various agent class files.

In [ ]:
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid
from mesa.visualization.modules import CanvasGrid
from mesa.visualization.ModularVisualization import ModularServer
from Tumor_cells import Tumor_cells
import random

In [ ]:

class M2(Agent):
    


## Instance
Initializes an M2 macrophage agent.
Sets up the M2 macrophage with its unique ID, position, and initial attributes such as probabilities of migration, death, and tumor growth support.
Initialization ensures each M2 macrophage is properly configured to simulate its role as a tumor-supporting agent in the model.


In [ ]:

    """
        Initializes an M2 macrophage agent.

        Args:
            unique_id (int): Unique identifier for the agent.
            model (Model): The model the agent belongs to.
            params (dict): Dictionary of model parameters.
    """
    def __init__(self, unique_id, position, model):
        super().__init__(unique_id, model)
        self.position = position
        self.prob_migrate = 0.4
        self.prob_death = 0.005
        self.prob_support_growth = 0.05
        self.alive = True


## Eat
Consumes a specified amount of nutrition from the environment.
Allows the M2 macrophage to interact with its environment by consuming resources, which may impact its behavior and survival.
Simulating resource consumption reflects the metabolic requirements of macrophages in the tumor microenvironment.



In [ ]:

    def eat(self, val):
        self.model.eat_nutrition(val)
    """
        Executes one step of the macrophage's behavior:
        - Checks if the macrophage dies.
        - Attempts to migrate to a neighboring cell.
        - Supports the growth of nearby tumor cells with a given probability.
    """


## Step
Executes one step of the M2 macrophage agent's behavior.
Defines the agent's lifecycle at each step, including:
- **Death:** The agent may die based on `prob_death`.
- **Migration:** The agent may move to a neighboring cell based on `prob_migrate`.
- **Support Tumor Growth:** The agent may enhance the proliferation and survival of nearby tumor cells with `prob_support_growth`.

This method encapsulates the decision-making process and simulates the M2 macrophage's role as a pro-tumor agent in the model.


In [ ]:

    def step(self):
        # Check if macrophage should die
        if not self.alive:
            return
        self.eat(5)
        if self.random.random() < self.prob_death:
            self.alive = False
            self.model.grid.remove_agent(self)
            self.model.schedule.remove(self)
            return
        if self.random.random() < self.prob_migrate:
            self.migrate()
        if self.random.random() < self.prob_support_growth:
            self.support_tumor_cells()



## Migration
Moves the M2 macrophage to a random neighboring cell if the new cell is empty.
Simulates the natural migration of macrophages, allowing them to interact with different parts of the tumor microenvironment.
Migration increases the macrophage's exposure to tumor cells, enhancing its ability to support tumor growth.


In [ ]:

        # Attempt to migrate
    def migrate(self):
        possible_steps = self.model.grid.get_neighborhood(self.pos, moore=True, include_center=False)
        
        # Filter only empty positions
        empty_positions = [pos for pos in possible_steps if self.model.grid.is_cell_empty(pos)]

        #Pick an empty position if there are any
        if len(empty_positions) > 0:
            new_position = self.random.choice(empty_positions)
            self.model.grid.move_agent(self, new_position)



## Support of tumor cells
Supports the growth of nearby tumor cells.
- Increases the proliferation probability of tumor cells.
- Reduces their death probability.
- Enhances angiogenesis intensity, promoting the formation of blood vessels to support tumor growth.
This method represents the tumor-supporting role of M2 macrophages, which is critical for simulating the tumor microenvironment's complexity.



In [ ]:
      
# Check neighbors for tumor cells
    def support_tumor_cells(self):
        neighbors = self.model.grid.get_neighbors(self.position, moore=True, include_center=False)
        for neighbor in neighbors:
            if isinstance(neighbor, Tumor_cells):
            # Support tumor growth with probability
                if self.random.random() < self.prob_support_growth:
                # Create a new tumor cell in a random neighboring position
                    neighbors = self.model.grid.get_neighborhood(neighbor.position, moore=True, include_center=False)
                    tumor_cells = [cell for cell in neighbors if isinstance(cell, Tumor_cells)]
                    if tumor_cells:
                        tumor_cell = self.random.choice(tumor_cells) 
                        tumor_cell.set_proliferation_prob(2, "proportion")
                        tumor_cell.set_death_prob(0,"value")
                    
                    for agent in tumor_cells:
                        agent.set_angiogenesis_intensity(1)
                        #print("M2 Supperoted TUMOR PROLIFERATIOn")
    
    """
        Moves the macrophage to a random neighboring position.
    """



## Random movement
Moves the M2 macrophage to a random neighboring position.
Allows for random movement of the macrophage, independent of its surrounding conditions.
Random movement introduces stochastic behavior, mimicking the variability observed in biological systems.


In [ ]:

    def random_move(self):
        possible_steps = self.model.grid.get_neighborhood(self.position, moore=True, include_center=False)
        new_position = self.random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)